# 評価方法

分類されていないデータを認識し、どれだけ正しくカテゴリごとに分類できるかを算出した「平均精度」の高さを競い合います。

今回、活用するデータはLSWMD_25519となります。
LSWMD_25519のFailureType項目が分類されていない状態のデータに対し、正しいFailureTypeカテゴリを分類するプログラムを作成し、その平均精度を算出します。
平均精度とは、カテゴリごとに正しく分類できる精度を平均した値です。カテゴリごとに算出した精度（Aが正しく分類された数/Aのデータ数）を足し、カテゴリ数で割ります。

公平な評価を実施するために、以下の制限を設けています。
1. 外部パッケージをインストールするためのセルとsolution関数の中身のみを編集すること
2. 校舎のiMac上で最後のセルの実行時間が15分未満であること　（%%timeitの出力結果を確認してください）

※気になる点がある場合、Discordで気軽にお問合せください。

In [1]:
import numpy as np # https://numpy.org/ja/
import pandas as pd # https://pandas.pydata.org/
from sklearn.model_selection import train_test_split

外部パッケージを使用する場合、以下の方法でインストールを実施してください。

In [2]:
# 必要な外部パッケージは、以下の内容を編集しインストールしてください
!pip install keras
!pip install opencv-python
!pip install tensorflow-cpu

以下のsolution関数のみ編集してください。

In [3]:
def solution(x_test_df, train_df):
    import numpy as np
    from keras.models import Sequential
    from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
    from keras.callbacks import EarlyStopping
    from sklearn.preprocessing import LabelEncoder
    from tensorflow.keras.utils import to_categorical
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau
    import cv2  # OpenCV をインポート
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Attention

    encoder = LabelEncoder()
    
    def rotate_image(image, angle):
        center = tuple(np.array(image.shape[1::-1]) / 2)
        rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
        return cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    
    def augment_images(images):
        '''
        訓練用のデータの array (3d array) を受け取り, 要素である 2d array を基に拡張された 2d array を 3d array に詰め直して返す関数。
        新たにデータ拡張のパターンを追加する場合は、以下に lamda 関数を追加することで追加可能である。
        '''
        return [
            operation(img)
            for img in images
            for operation in [
                # 実行されるデータ拡張手法
                lambda x: x,
                lambda x: rotate_image(x, 90),
                lambda x: rotate_image(x, 180),
                lambda x: rotate_image(x, 270),
                lambda x: cv2.flip(x, 0),
                lambda x: cv2.flip(rotate_image(x, 90), 0),
                lambda x: cv2.flip(rotate_image(x, 180), 0),
                lambda x: cv2.flip(rotate_image(x, 270), 0)
            ]
        ]

    height, width = 43, 43
    channels = 1
    target_size = (height, width)  # ターゲットサイズを (幅, 高さ) の順で設定
    input_shape = (height, width, channels) # input shape を設定
    
    # オリジナルの画像をリサイズ
    resized_images = [cv2.resize(img, target_size[::-1], interpolation=cv2.INTER_LINEAR) for img in train_df['waferMap']]
    
    # アフィン変換を適用してデータを増やす
    augmented_images = augment_images(resized_images)
    
    # NumPy配列に変換
    X_train = np.array(augmented_images).reshape(-1, height, width, channels)
    
    # ラベルの準備（ラベルも8倍に増やす）
    encoded_Y = encoder.fit_transform(train_df['failureType'])
    y_train = to_categorical(encoded_Y)
    y_train = np.repeat(y_train, 8, axis=0)  # 同じラベルを8回繰り返す

    model = Sequential()
    input_layer = Input(shape=input_shape)
    
    # 畳み込み層とプーリング層
    x = Conv2D(32, (5, 5), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(32, (5, 5), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    
    # Attentionレイヤーの追加
    attention_output = Attention()([x, x])
    
    # Flattenレイヤーの追加
    x = Flatten()(attention_output)
    
    # 全結合層
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    output_layer = Dense(y_train.shape[1], activation='softmax')(x)
    
    # モデルの定義
    model = Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=20)

    # 学習率を自動的に減らすコールバックを作成
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.01)
    
    model.fit(X_train, y_train, epochs=8, batch_size=64, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

    resized_images_test = [ cv2.resize(img, target_size[::-1], interpolation=cv2.INTER_LINEAR) for img in x_test_df['waferMap'] ]
    
    X_test = np.array(resized_images_test).reshape(-1, height, width, channels)
    
    y_pred = np.argmax(model.predict(X_test), axis=-1)

    return pd.DataFrame({'failureType': encoder.inverse_transform(y_pred)}, index=x_test_df.index)

solution関数は以下のように活用され、平均精度を計算します。

In [ ]:
%%timeit -r 1 -n 1

# データのインポート
df = pd.read_pickle("../input/LSWMD_25519.pkl")

# テスト用と学習用のデータを作成（テストする際は、random_stateの値などを編集してみてください）
train_df, test_df = train_test_split(df, stratify=df['failureType'], test_size=0.10, random_state=42)

y_test_df = test_df[['failureType']]
x_test_df = test_df.drop(columns=['failureType'])

# solution関数を実行
user_result_df = solution(x_test_df, train_df)

average_accuracy = 0
# ユーザーの提出物のフォーマット確認
if type(y_test_df) == type(user_result_df) and y_test_df.shape == user_result_df.shape:
    # 平均精度の計算
    accuracies = {}
    for failure_type in df['failureType'].unique():
        y_test_df_by_failure_type = y_test_df[y_test_df['failureType'] == failure_type]
        user_result_df_by_failure_type = user_result_df[y_test_df['failureType'] == failure_type]
        matching_rows = (y_test_df_by_failure_type == user_result_df_by_failure_type).all(axis=1).sum()
        accuracies[failure_type] = matching_rows / len(y_test_df_by_failure_type)
        print(f"不良タイプ {failure_type} の正答率：{accuracies[failure_type] * 100:.2f}%")
    
    average_accuracy = sum(accuracies.values()) / len(accuracies)

print(f"平均精度：{average_accuracy * 100:.2f}%")

2023-11-20 06:26:55.746916: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/8
2297/2297 [==============================] - 103s 44ms/step - loss: 0.4729 - accuracy: 0.8226 - val_loss: 0.3807 - val_accuracy: 0.8583 - lr: 0.0010
Epoch 2/8
2297/2297 [==============================] - 103s 45ms/step - loss: 0.3409 - accuracy: 0.8709 - val_loss: 0.6437 - val_accuracy: 0.7776 - lr: 0.0010
Epoch 3/8
2297/2297 [==============================] - 99s 43ms/step - loss: 0.2951 - accuracy: 0.8889 - val_loss: 0.3199 - val_accuracy: 0.8766 - lr: 0.0010
Epoch 4/8
2297/2297 [==============================] - 101s 44ms/step - loss: 0.2661 - accuracy: 0.8990 - val_loss: 0.3756 - val_accuracy: 0.8680 - lr: 0.0010
Epoch 5/8
 322/2297 [===>..........................] - ETA: 1:27 - loss: 0.2397 - accuracy: 0.9097